In [ ]:
!pip install rasterio
!pip install geopandas
!pip install pyproj
!pip install rasterio
!pip install geopandas
!pip install pyproj
!pip install scikit-learn
!pip install scipy
!pip install scikit-learn
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.1 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd

# Load shapefile
input_shapefile = "/content/Stream_sediment_clip.shp"  # Update with your file
gdf = gpd.read_file(input_shapefile)

# Reproject to EPSG:32643 (UTM zone 43N)
if gdf.crs.to_epsg() != 32643:
    gdf = gdf.to_crs(epsg=32643)

# View columns to confirm
print(gdf.columns)


Index(['gid', 'objectid', 'sampleno', 'X', 'Y', 'Si02__', 'Al2O3__', 'Fe2O3__',
       'TiO2__', 'CaO__', 'MgO__', 'MnO__', 'Na2O__', 'K2O__', 'P2O5__', 'LOI',
       'Ba_ppm', 'Ga_ppm', 'Sc_ppm', 'V_ppm', 'Th_ppm', 'Pb_ppm', 'Ni_ppm',
       'Co_ppm', 'Rb_ppm', 'Sr_ppm', 'Y_ppm', 'Zr_ppm', 'Nb_ppm', 'Cr_ppm',
       'Cu_ppm', 'Zn_ppm', 'Au_ppb', 'Li_ppm', 'Cs_ppm', 'As_ppm', 'Sb_ppm',
       'Bi_ppm', 'Se_ppm', 'Ag_ppm', 'Cd_ppm', 'Hg_ppb', 'Be_ppm', 'Ge_ppm',
       'Mo_ppm', 'Sn_ppm', 'La_ppb', 'Ce_ppb', 'Pr_ppb', 'Nd_ppb', 'Sm_ppb',
       'Eu_ppb', 'Tb_ppb', 'Gd_ppb', 'Dy_ppb', 'Ho_ppb', 'Er_ppb', 'Tm_ppb',
       'Yb_ppb', 'Lu_ppb', 'Hf_ppm', 'Ta_ppm', 'W_ppm', 'U_ppm', 'Pt_ppb',
       'Pd_ppb', 'In_ppm', 'F_ppm', 'Te_ppm', 'Tl_ppm', 'geometry'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# Select geochemical columns
columns = ['Ni_ppm', 'Cu_ppm', 'Cr_ppm', 'MgO__', 'Fe2O3__', 'Zr_ppm']
df_clean = gdf[columns].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean)

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

# Store PCA scores
gdf.loc[df_clean.index, 'PCA1'] = pca_result[:, 0]
gdf.loc[df_clean.index, 'PCA2'] = pca_result[:, 1]

# Coordinates for interpolation
coords = np.array([(pt.x, pt.y) for pt in gdf.loc[df_clean.index].geometry])
values = gdf.loc[df_clean.index, 'PCA1'].values


In [ ]:
from scipy.spatial import cKDTree

def idw_kdtree(coords, values, xi, yi, power=2, k=12):
    grid_x, grid_y = np.meshgrid(xi, yi)
    grid_points = np.column_stack((grid_x.ravel(), grid_y.ravel()))

    tree = cKDTree(coords)
    dists, idxs = tree.query(grid_points, k=k)

    dists[dists == 0] = 1e-10  # Avoid division by zero
    weights = 1 / dists**power
    z = np.sum(weights * values[idxs], axis=1) / np.sum(weights, axis=1)

    return z.reshape(grid_x.shape), grid_x, grid_y


In [ ]:
import rasterio
from rasterio.transform import from_origin

# Define 30m grid
res = 30
minx, miny, maxx, maxy = gdf.total_bounds
xi = np.arange(minx, maxx, res)
yi = np.arange(maxy, miny, -res)  # y in descending order for raster

# Run IDW interpolation
idw_grid, grid_x, grid_y = idw_kdtree(coords, values, xi, yi)

# Save to GeoTIFF
transform = from_origin(minx, maxy, res, res)
with rasterio.open(
    "PCA1_IDW_30m.tif", "w",
    driver="GTiff",
    height=idw_grid.shape[0],
    width=idw_grid.shape[1],
    count=1,
    dtype=idw_grid.dtype,
    crs="EPSG:32643",
    transform=transform
) as dst:
    dst.write(idw_grid, 1)


In [ ]:
from pyproj import Transformer

# Transform to decimal degrees (EPSG:4326)
transformer = Transformer.from_crs("EPSG:32643", "EPSG:4326", always_xy=True)
gdf["Longitude"], gdf["Latitude"] = transformer.transform(gdf.geometry.x.values, gdf.geometry.y.values)

# Save CSV
gdf[['Longitude', 'Latitude', 'PCA1', 'PCA2']].dropna().to_csv("PCA_scores_with_latlon.csv", index=False)
